In [7]:
import boto3
s3 = boto3.client('s3')
bucket_name = 'moviereco'
data_key = 'tmdb-movie-metadata/'
credits_file_name = 'tmdb-movie-metadata/tmdb_5000_credits.csv'
movies_file_name = 'tmdb-movie-metadata/tmdb_5000_movies.csv'
# List objects in the bucket
response = s3.list_objects(Bucket=bucket_name, Prefix=data_key)
for content in response['Contents']:
    print(content['Key'])

tmdb-movie-metadata/tmdb_5000_credits.csv
tmdb-movie-metadata/tmdb_5000_movies.csv


In [3]:
!pip install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.0/138.0 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 90.4 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 110.5 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.13.3
    Uninstalling tokenizers-0.13.3:
      Successfully uninstalled tokenizers-0.13.3
  Attempting uninstall: transformers
    Found existing installation: transformers 4.31.0
    Uninstalling transformers-4.31.0:
      Successfully uninstalled transformers-4.31.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
autogluon-multimodal 0.8.2 requires transformers[sentencepiece]<4.32.0,>=4.31.0, but you have transformers 4.40.1 which is incompatible.


In [4]:
from sentence_transformers import SentenceTransformer, util
import pandas as pd
import ast

In [6]:
def read_csv(client, bucket_name, file_name, date_cols=None):
    obj = client.get_object(Bucket=bucket_name, Key=file_name) 
    return pd.read_csv(obj['Body'],parse_dates=date_cols)

In [11]:
movie = read_csv(s3, bucket_name, movies_file_name)
movie[['genres','keywords','production_companies','production_countries','spoken_languages']] = movie[['genres','keywords','production_companies','production_countries','spoken_languages']].map(lambda x : ast.literal_eval(str(x)))

def get_data(x, cols, dict):
    for col in cols:
        for i in range(len(x[col])):
            for j in range(len(x[col][i])):
                x[col][i][j] = x[col][i][j][dict]
    return x

In [12]:
def get_data(x, cols, dict):
    for col in cols:
        for i in range(len(x[col])):
            for j in range(len(x[col][i])):
                x[col][i][j] = x[col][i][j][dict]
    return x

In [13]:
movie = get_data(movie, ['genres','keywords','production_companies','production_countries','spoken_languages'],'name')

movie[['budget','id','popularity','revenue','runtime','vote_average','vote_count']] = movie[['budget','id','popularity','revenue','runtime','vote_average','vote_count']].apply(pd.to_numeric, errors = 'coerce')

movie.head(5)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[Action, Adventure, Fantasy, Science Fiction]",http://www.avatarmovie.com/,19995,"[culture clash, future, space war, space colon...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[Ingenious Film Partners, Twentieth Century Fo...","[United States of America, United Kingdom]",2009-12-10,2787965087,162.0,"[English, Español]",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[Adventure, Fantasy, Action]",http://disney.go.com/disneypictures/pirates/,285,"[ocean, drug abuse, exotic island, east india ...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[Walt Disney Pictures, Jerry Bruckheimer Films...",[United States of America],2007-05-19,961000000,169.0,[English],Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[Action, Adventure, Crime]",http://www.sonypictures.com/movies/spectre/,206647,"[spy, based on novel, secret agent, sequel, mi...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[Columbia Pictures, Danjaq, B24]","[United Kingdom, United States of America]",2015-10-26,880674609,148.0,"[Français, English, Español, Italiano, Deutsch]",Released,A Plan No One Escapes,Spectre,6.3,4466
3,250000000,"[Action, Crime, Drama, Thriller]",http://www.thedarkknightrises.com/,49026,"[dc comics, crime fighter, terrorist, secret i...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[Legendary Pictures, Warner Bros., DC Entertai...",[United States of America],2012-07-16,1084939099,165.0,[English],Released,The Legend Ends,The Dark Knight Rises,7.6,9106
4,260000000,"[Action, Adventure, Science Fiction]",http://movies.disney.com/john-carter,49529,"[based on novel, mars, medallion, space travel...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,[Walt Disney Pictures],[United States of America],2012-03-07,284139100,132.0,[English],Released,"Lost in our world, found in another.",John Carter,6.1,2124


In [14]:
credits = read_csv(s3, bucket_name, credits_file_name)

In [15]:
credits[['cast', 'crew']] = credits[['cast', 'crew']].map(lambda x : ast.literal_eval(str(x)))

credits = get_data(credits, ['cast', 'crew'],'name')

credits.head(5)

,movie_id,title,cast,crew
0,19995,Avatar,"[Sam Worthington, Zoe Saldana, Sigourney Weave...","[Stephen E. Rivkin, Rick Carter, Christopher B..."
1,285,Pirates of the Caribbean: At World's End,"[Johnny Depp, Orlando Bloom, Keira Knightley, ...","[Dariusz Wolski, Gore Verbinski, Jerry Bruckhe..."
2,206647,Spectre,"[Daniel Craig, Christoph Waltz, Léa Seydoux, R...","[Thomas Newman, Sam Mendes, Anna Pinnock, John..."
3,49026,The Dark Knight Rises,"[Christian Bale, Michael Caine, Gary Oldman, A...","[Hans Zimmer, Charles Roven, Christopher Nolan..."
4,49529,John Carter,"[Taylor Kitsch, Lynn Collins, Samantha Morton,...","[Andrew Stanton, Andrew Stanton, John Lasseter..."


In [16]:
movie = pd.merge(movie, credits[['movie_id','cast', 'crew']],  left_on= "id", right_on = "movie_id", how = "left")
movie['overview'] = movie['overview'].astype(str)

In [17]:
model = SentenceTransformer('all-MiniLM-L6-v2')
overview_embeddings = model.encode(movie['overview'])
overview_cos_sim = util.cos_sim(overview_embeddings, overview_embeddings)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [18]:
def recommender(movie_name):
    result = pd.concat([movie["original_title"],
                    pd.DataFrame(overview_cos_sim[:,movie[movie["original_title"] == movie_name].index].numpy(), columns=['Overview'])],axis = 1)
    result = result[result["Overview"] != 1]
    result = result.sort_values('Overview', ascending= False).head(10).reset_index(drop =  True)
    return result

In [23]:
print(recommender("Spectre"))

            original_title  Overview
0                  Spectre  1.000000
1    Never Say Never Again  0.627305
2                Octopussy  0.620293
3     The Living Daylights  0.606730
4                  Skyfall  0.604230
5  The World Is Not Enough  0.591345
6         Live and Let Die  0.555852
7    From Russia with Love  0.540722
8       For Your Eyes Only  0.528804
9                GoldenEye  0.525823


In [20]:
print(recommender("Memento"))

              original_title  Overview
0                    Gothika  0.547444
1              Drowning Mona  0.490740
2               The Majestic  0.482932
3  A Nightmare on Elm Street  0.481691
4     El secreto de sus ojos  0.477472
5        The Next Three Days  0.476633
6                     Torque  0.476069
7             Enter the Void  0.473206
8                 Salton Sea  0.468047
9               Another Year  0.466985
